In [ ]:
# !pip install pandas==1.1.3

In [ ]:
! pip install shap
! pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 564 kB 7.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 108 kB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Data Wrangling
import pandas as pd
from pandas import Series, DataFrame
import numpy as np

# Visualization
import matplotlib.pylab as plt
from matplotlib import font_manager, rc
import seaborn as sns
%matplotlib inline

# Preprocessing & Feature Engineering
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.feature_selection import SelectPercentile

# Hyperparameter Optimization
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

# Modeling
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVC
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.base import RegressorMixin
from sklearn.ensemble import ExtraTreesRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostClassifier
from sklearn.linear_model import Ridge

# Evaluation
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

# Utility
import os
import time
import random
import warnings; warnings.filterwarnings("ignore")
from IPython.display import Image
import pickle
from tqdm import tqdm
import platform
from itertools import combinations
from scipy.stats.mstats import gmean

from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow import keras

In [ ]:
df_train = pd.read_csv('/content/drive/MyDrive/강민수+이효빈+한윤지/input/X_train.csv',encoding='cp949')
df_test = pd.read_csv('/content/drive/MyDrive/강민수+이효빈+한윤지/input/X_test.csv',encoding='cp949')
df = pd.concat([df_train, df_test])
df = df.iloc[:,1:]
y_train = pd.read_csv('/content/drive/MyDrive/강민수+이효빈+한윤지/input/y_train.csv').group
IDtest = df_test.custid.unique()

In [ ]:
# X_train = pd.read_csv(os.path.abspath("../input")+'/1,2등,우리_selection_3.csv',encoding = 'utf-8')
# del X_train['Unnamed: 0']
# X_test = pd.read_csv(os.path.abspath("../input")+'/1,2등,우리_selection_3_te.csv',encoding = 'utf-8')
# del X_test['Unnamed: 0']

### Word2Vec

2등팀

In [ ]:
# 이름 비슷한거 통일
df_train.part_nm = df_train.part_nm.replace('여성캐쥬얼','여성캐주얼').replace('스포츠캐쥬얼','스포츠캐주얼').replace(
                    '가정용품파트','가정용품').replace('생식품파트','생식품').replace('공산품파트','공산품').replace(
                    '로얄부틱','로얄부띠끄').replace('잡화파트','잡화')
df_test.part_nm = df_test.part_nm.replace('여성캐쥬얼','여성캐주얼').replace('스포츠캐쥬얼','스포츠캐주얼').replace(
                    '가정용품파트','가정용품').replace('생식품파트','생식품').replace('공산품파트','공산품').replace(
                    '로얄부틱','로얄부띠끄').replace('잡화파트','잡화')

In [ ]:
import random

df_train2=df_train.reset_index().drop('index',axis=1).copy()
df_train2=df_train2.reset_index()
df_train2['goodcd']=df_train2['goodcd'].astype('str')
train_data=list(df_train2.groupby('custid')['goodcd'].unique())

In [ ]:
def oversample(x, n):
    lst = []
    for i in x:
        tmp = []
        for j in range(n):
            random.shuffle(i)
            tmp += list(i)
            lst.append(tmp)
    return lst

In [ ]:
w2v_input = oversample(train_data, 5)

In [ ]:
from gensim.models import word2vec

w2v = word2vec.Word2Vec(sentences = w2v_input, size = 100, window = 3, min_count = 1, sg = 1)

In [ ]:
train_mean_vector = []
for words in tqdm(train_data):
    tmp = np.zeros(100)
    cnt = 0
    for word in words:
        try:
            tmp += w2v[word]
            cnt += 1
        except:
            pass
    tmp /= cnt
    train_mean_vector.append(tmp)
train_mean_vector = np.array(train_mean_vector)

100%|██████████| 21587/21587 [00:02<00:00, 8150.03it/s]


In [ ]:
train_mean_vector = pd.DataFrame(train_mean_vector)
train_mean_vector

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.227112,-0.082365,0.091567,0.071874,0.059538,-0.011250,-0.039674,0.159170,0.004575,0.061330,...,-0.004261,-0.128048,0.062953,-0.028687,-0.293057,0.202978,-0.000467,-0.040181,0.055257,-0.087079
1,-0.146752,-0.177444,-0.074010,0.157410,0.193644,0.080630,-0.083824,0.465267,0.212363,0.100952,...,0.120503,0.021155,-0.115317,-0.096885,-0.090171,-0.007919,0.101326,-0.049732,0.181898,-0.226223
2,-0.124412,-0.070367,0.044984,0.144362,0.101615,-0.231669,-0.010185,0.113517,0.186647,-0.072060,...,0.139690,0.067315,-0.260695,-0.025189,-0.152981,0.176988,-0.044958,-0.312435,0.169983,-0.410159
3,-0.451491,-0.111339,-0.263775,-0.112683,-0.068191,0.042263,-0.387242,0.021852,0.211160,0.396781,...,-0.132978,0.024882,0.094970,0.467005,-0.352310,0.155050,-0.028698,-0.172806,-0.088059,-0.186899
4,-0.221306,-0.078386,-0.119899,0.154876,0.124777,-0.093637,-0.102438,0.208460,0.030589,0.094041,...,0.184824,-0.109004,-0.023989,-0.052933,-0.083171,0.159132,0.034167,0.001636,0.096761,-0.059588
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21582,-0.104943,-0.063712,-0.021746,0.144144,0.198179,0.042125,-0.082595,0.210219,0.146303,-0.088166,...,0.022426,-0.177408,0.013953,-0.101606,-0.151281,0.201129,0.007405,-0.002606,0.114058,0.010674
21583,-0.321024,-0.077736,0.115087,-0.012513,0.138176,0.069123,-0.127209,0.092518,0.141934,-0.013174,...,0.104615,-0.034216,-0.155191,0.162371,-0.086092,0.165290,-0.053577,-0.064319,0.126721,-0.210660
21584,-0.141488,0.132592,0.126661,0.384647,0.198752,-0.032268,0.044426,0.300116,0.177664,-0.179719,...,0.204368,-0.076751,0.036331,-0.019966,-0.067264,0.112788,0.044271,-0.089934,0.327220,-0.208325
21585,-0.148908,-0.046165,0.204213,-0.089428,0.056945,-0.170570,-0.154547,0.339079,0.067009,0.123997,...,-0.014709,-0.035297,0.125019,0.286948,-0.217728,0.247719,-0.112078,-0.045394,0.025290,-0.317357


In [ ]:
# income_occyp_type, income_age_type, edu_occyp_type, edu_income_type, family_income_type

import random

df_test2=df_test.reset_index().drop('index',axis=1).copy()
df_test2=df_test2.reset_index()
df_test2['goodcd']=df_test2['goodcd'].astype('str')
test_data=list(df_test2.groupby('custid')['goodcd'].unique())

In [ ]:
test_mean_vector = []
for words in tqdm(test_data):
    tmp = np.zeros(100)
    cnt = 0
    for word in words:
        try:
            tmp += w2v[word]
            cnt += 1
        except:
            pass
    tmp /= cnt
    test_mean_vector.append(tmp)
test_mean_vector = np.array(test_mean_vector)

100%|██████████| 14380/14380 [00:01<00:00, 7907.84it/s]


In [ ]:
test_mean_vector = pd.DataFrame(test_mean_vector)
test_mean_vector

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.055104,-0.026126,-0.099940,0.115738,0.155492,-0.084827,-0.183163,0.280413,0.020677,0.057614,...,0.042137,-0.152185,0.100372,-0.152181,-0.103009,-0.049390,0.025316,-0.103630,0.175685,-0.019231
1,-0.049329,-0.081267,-0.046575,0.192381,0.032564,-0.097298,-0.191535,0.215682,0.023859,-0.088198,...,0.072455,-0.143929,0.007459,-0.034106,-0.134299,0.145655,0.036979,-0.057481,0.097145,-0.089601
2,-0.292738,-0.074535,0.010573,0.002738,0.323695,-0.137852,0.057418,0.000051,0.184704,0.098462,...,0.075898,-0.032967,-0.129256,0.012336,-0.121442,0.125344,0.046489,0.005342,0.279223,-0.221342
3,-0.270631,-0.279013,0.095705,0.081627,0.169003,0.046655,-0.151934,0.112165,0.148010,-0.045940,...,-0.032019,-0.016186,-0.182576,0.013651,-0.099863,0.086874,0.122635,0.055704,-0.031422,-0.092013
4,-0.318685,-0.267141,0.100405,0.025648,0.208244,-0.029563,-0.136242,0.086938,0.053532,0.070884,...,0.119453,-0.055152,-0.172221,0.078093,-0.243153,0.070046,-0.029125,-0.074108,0.206081,-0.263496
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14375,-0.185515,-0.039375,-0.166546,0.141493,0.093266,-0.275151,-0.146123,0.176989,-0.128264,-0.058078,...,-0.110445,-0.077137,-0.192369,-0.182169,-0.133109,0.174365,-0.186286,0.107519,0.206699,0.221853
14376,-0.401652,-0.185662,0.159575,-0.000964,0.217943,-0.054304,-0.074388,0.020814,0.081924,0.076464,...,0.181836,0.036485,-0.134630,-0.037853,-0.147302,0.118502,-0.132549,-0.073138,0.205586,-0.156212
14377,-0.271361,-0.037076,-0.026019,-0.005503,0.302102,-0.091478,0.063985,0.215046,0.107284,0.018328,...,0.039851,0.139243,-0.072743,0.044719,-0.077340,0.066632,-0.131082,-0.161403,0.111325,-0.020892
14378,-0.144051,-0.235766,-0.063933,0.127514,0.322950,-0.237006,-0.210958,0.115926,0.028699,-0.070040,...,0.024666,-0.115784,-0.082658,0.006334,-0.136771,0.164518,-0.267907,0.019788,0.392267,-0.072219


In [ ]:
# income_occyp_type, income_age_type, edu_occyp_type, edu_income_type, family_income_type

import random

df_train2=df_train.reset_index().drop('index',axis=1).copy()
df_train2=df_train2.reset_index()
train_data=list(df_train2.groupby('custid')['brd_nm'].unique())

In [ ]:
def oversample(x, n):
    lst = []
    for i in x:
        tmp = []
        for j in range(n):
            random.shuffle(i)
            tmp += list(i)
            lst.append(tmp)
    return lst

In [ ]:
w2v_input = oversample(train_data, 5)

In [ ]:
from gensim.models import word2vec

w2v = word2vec.Word2Vec(sentences = w2v_input, size = 100, window = 3, min_count = 1, sg = 1)

In [ ]:
train_mean_vector_2 = []
for words in tqdm(train_data):
    tmp = np.zeros(100)
    cnt = 0
    for word in words:
        try:
            tmp += w2v[word]
            cnt += 1
        except:
            pass
    tmp /= cnt
    train_mean_vector_2.append(tmp)
train_mean_vector_2 = np.array(train_mean_vector_2)

100%|██████████| 21587/21587 [00:02<00:00, 8785.19it/s]


In [ ]:
train_mean_vector_2 = pd.DataFrame(train_mean_vector_2)
train_mean_vector_2

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.171042,-0.042435,0.053787,0.121208,0.081139,-0.052773,0.173243,-0.017650,0.204878,-0.203825,...,0.186285,0.018814,0.071283,0.212826,-0.099886,0.075576,-0.040172,0.417430,-0.054138,0.114116
1,-0.211081,-0.071296,0.059165,0.091273,-0.012659,0.010319,0.035598,0.111081,0.139748,-0.230237,...,0.118564,0.018122,0.026043,0.169185,-0.062341,-0.013679,0.087478,0.338534,-0.122886,0.151681
2,-0.187127,-0.003234,-0.049938,0.134284,-0.025323,0.014952,-0.026341,0.082052,0.064136,-0.224375,...,0.038932,0.096927,0.043879,0.151363,0.025464,0.083853,0.027301,0.271923,-0.056255,0.165875
3,-0.207697,-0.078818,-0.131111,0.451450,-0.121539,-0.125933,0.127812,0.001877,-0.154775,-0.315713,...,-0.100736,-0.121997,-0.004219,0.154130,-0.419432,0.085016,0.217536,0.293995,0.190447,0.328341
4,-0.136641,-0.028137,0.136033,0.155103,0.034971,0.057041,0.018174,-0.030057,0.092012,-0.163613,...,0.082245,0.000384,0.073835,0.180011,-0.062789,0.044405,0.003149,0.322643,-0.049324,0.133654
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21582,-0.034707,-0.071994,0.064886,0.229905,0.008170,-0.012474,0.159285,0.022992,0.097664,-0.236414,...,0.136338,0.046129,0.077834,0.224871,-0.028780,-0.007095,0.029899,0.393389,0.002080,0.225115
21583,-0.214221,-0.074432,-0.023867,0.197011,-0.060777,0.085133,-0.005576,-0.007883,0.177519,-0.330054,...,0.200140,0.055232,0.055666,0.240469,-0.115496,-0.025805,0.006722,0.423182,-0.042651,0.147674
21584,-0.081176,-0.039269,0.163191,0.159261,0.104319,-0.101981,0.048170,0.087364,-0.002058,-0.337110,...,0.099245,0.059335,-0.002843,0.230369,-0.074533,0.032689,-0.019700,0.326561,0.001237,0.185259
21585,-0.108931,-0.131814,0.005433,0.188403,0.058268,-0.056043,-0.025020,-0.137033,0.119430,-0.298620,...,0.110601,-0.011932,0.060046,0.110860,-0.064051,-0.032800,0.061711,0.287392,-0.089301,0.115103


In [ ]:
# income_occyp_type, income_age_type, edu_occyp_type, edu_income_type, family_income_type

import random

df_test2=df_test.reset_index().drop('index',axis=1).copy()
df_test2=df_test2.reset_index()
test_data=list(df_test2.groupby('custid')['brd_nm'].unique())

In [ ]:
test_mean_vector_2 = []
for words in tqdm(test_data):
    tmp = np.zeros(100)
    cnt = 0
    for word in words:
        try:
            tmp += w2v[word]
            cnt += 1
        except:
            pass
    tmp /= cnt
    test_mean_vector_2.append(tmp)
test_mean_vector_2 = np.array(test_mean_vector_2)

100%|██████████| 14380/14380 [00:01<00:00, 8664.29it/s]


In [ ]:
test_mean_vector_2 = pd.DataFrame(test_mean_vector_2)
test_mean_vector_2

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.153277,-0.021223,0.031757,0.182027,0.016301,-0.057399,0.148926,-0.002991,0.216047,-0.141495,...,0.066861,0.060338,0.100833,0.171925,-0.092280,0.071031,-0.006421,0.300475,-0.136870,0.155341
1,-0.137334,-0.038323,0.106908,0.150877,0.018184,0.026782,0.122723,0.006047,0.107534,-0.146305,...,0.119091,0.032104,0.092968,0.223858,-0.037115,-0.012824,0.031753,0.305346,-0.043422,0.190609
2,-0.119863,0.007706,0.052553,0.173968,-0.023382,0.030007,0.056164,-0.023233,0.097748,-0.276495,...,0.143126,0.098805,0.106304,0.259939,-0.104896,-0.023302,-0.047690,0.464689,-0.119205,0.151381
3,-0.076550,0.003050,0.020693,0.218640,-0.105261,0.107855,0.105109,0.106880,0.146649,-0.263513,...,0.322794,0.098388,0.106431,0.166974,-0.134004,0.090691,-0.011165,0.483272,0.023306,0.165402
4,-0.111497,0.042054,0.059122,0.096521,0.006421,0.087624,-0.005743,-0.018342,0.106978,-0.219889,...,0.182044,0.034673,0.098362,0.218090,-0.127153,-0.021604,0.034519,0.380326,-0.092742,0.166469
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14375,-0.157706,-0.015357,0.342208,0.132239,-0.071103,0.000720,0.160009,-0.002117,0.002235,-0.227517,...,0.184228,0.048222,0.221367,0.101987,0.045640,-0.009539,0.103143,0.391234,0.008178,0.194317
14376,-0.258323,-0.007725,0.082550,0.175321,-0.018230,0.077933,-0.019590,-0.125174,0.225288,-0.273064,...,0.193809,-0.165190,0.012933,0.148510,-0.061024,0.076820,0.011436,0.547235,-0.102322,0.123886
14377,-0.136869,0.146268,-0.136416,0.209393,0.040465,-0.000568,0.150568,0.040284,0.010643,-0.319716,...,0.207601,-0.067105,0.038692,0.271852,0.070693,0.036592,0.199023,0.301681,0.018657,0.196631
14378,-0.068841,0.047714,0.044358,0.075937,-0.106262,-0.013256,-0.040625,0.098848,0.078602,-0.206665,...,0.003806,0.141832,0.081713,0.122522,0.002425,-0.033829,0.087020,0.300828,-0.066259,0.231534


1등팀
- bnd_nm은 2등조꺼에 있어서 그거 제외하고 

In [ ]:
# 구매 정보 관련
df_train['customer_info'] = df_train['corner_nm'].astype(str) + '_' + df_train['pc_nm'].astype(str) + '_' + df_train['part_nm'].astype(str)

df_test['customer_info'] = df_test['corner_nm'].astype(str) + '_' + df_test['pc_nm'].astype(str) + '_' + df_test['part_nm'].astype(str)

In [ ]:
import random

train_data = list(df_train.groupby('custid')['customer_info'].unique())
test_data = list(df_test.groupby('custid')['customer_info'].unique())

In [ ]:
def oversample(x, n):
    lst = []
    for i in x:
        tmp = []
        for j in range(n):
            random.shuffle(i)
            tmp += list(i)
            lst.append(tmp)
    return lst

In [ ]:
w2v_input = oversample(train_data, 5)

In [ ]:
from gensim.models import word2vec

w2v = word2vec.Word2Vec(sentences = w2v_input, size = 100, window = 3, min_count = 1, sg = 1)

In [ ]:
train_mean_vector_3 = []
for words in tqdm(train_data):
    tmp = np.zeros(100)
    cnt = 0
    for word in words:
        try:
            tmp += w2v[word]
            cnt += 1
        except:
            pass
    tmp /= cnt
    train_mean_vector_3.append(tmp)
train_mean_vector_3 = np.array(train_mean_vector_3)

100%|██████████| 21587/21587 [00:03<00:00, 6955.99it/s]


In [ ]:
train_mean_vector_3 = pd.DataFrame(train_mean_vector_3)
train_mean_vector_3

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.081916,0.247949,-0.116024,-0.353530,0.043546,0.261861,0.084367,0.001700,0.298787,0.154734,...,-0.262960,0.101225,-0.239781,0.215696,0.027970,0.075303,-0.053517,-0.188997,-0.186852,-0.106533
1,-0.012731,0.105487,-0.211561,-0.251680,-0.007592,0.336293,0.119320,-0.069379,0.304972,0.164325,...,-0.209561,0.005786,-0.186360,0.110726,-0.075476,0.124248,-0.107861,-0.044990,-0.253490,0.028441
2,0.131316,0.033263,-0.097964,-0.252542,-0.147555,0.258610,0.172981,-0.023102,0.297879,0.100698,...,-0.249242,-0.079477,-0.256175,-0.036121,-0.034074,0.164272,-0.228131,0.005472,-0.135282,0.045257
3,0.082842,0.110540,-0.043904,-0.188123,0.218764,0.133300,0.053332,-0.068794,0.259917,0.209173,...,-0.418847,0.271500,-0.296268,0.261197,-0.160166,0.059526,-0.004814,-0.131114,-0.143214,-0.009352
4,0.164892,0.107958,-0.139155,-0.336182,0.059948,0.252072,0.012862,0.066418,0.172878,0.174747,...,-0.168027,0.101039,-0.290378,0.195131,0.113169,0.133269,-0.113495,-0.027646,-0.134583,-0.143414
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21582,0.100892,0.136813,-0.179065,-0.370323,0.059513,0.141585,-0.014522,0.007909,0.211376,0.143409,...,-0.223827,0.172967,-0.257215,0.200648,-0.029096,0.147529,-0.136134,-0.158053,-0.187146,-0.046167
21583,0.181889,0.177981,-0.163943,-0.336036,0.170088,0.151061,-0.038639,0.015799,0.083894,0.114651,...,-0.327377,0.207489,-0.046475,0.208450,-0.119683,0.140253,0.001008,0.056070,-0.159214,0.032968
21584,0.137237,0.135269,-0.150670,-0.358171,0.063380,0.174995,-0.006938,-0.090479,0.230515,0.091497,...,-0.174476,0.174839,-0.168144,0.119948,0.013451,0.187458,-0.135146,-0.079971,-0.108782,-0.082825
21585,0.062165,0.261305,-0.211095,-0.286320,0.107803,0.267551,0.140783,-0.028420,0.181976,0.122747,...,-0.248715,0.302618,-0.122395,0.192601,-0.143187,0.074089,0.047767,0.005412,-0.234975,0.004053


In [ ]:
test_mean_vector_3 = []
for words in tqdm(test_data):
    tmp = np.zeros(100)
    cnt = 0
    for word in words:
        try:
            tmp += w2v[word]
            cnt += 1
        except:
            pass
    tmp /= cnt
    test_mean_vector_3.append(tmp)
test_mean_vector_3 = np.array(test_mean_vector_3)

100%|██████████| 14380/14380 [00:01<00:00, 7324.13it/s]


In [ ]:
test_mean_vector_3 = pd.DataFrame(test_mean_vector_3)
test_mean_vector_3

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.092291,0.121309,-0.182500,-0.401744,0.070245,0.245633,0.003181,0.037196,0.212337,0.129673,...,-0.197753,0.078910,-0.279538,0.195435,0.073110,0.105182,-0.113696,-0.083493,-0.097533,-0.132629
1,0.112761,0.104189,-0.159939,-0.309001,0.135844,0.196710,-0.023118,0.003928,0.199915,0.121126,...,-0.278052,0.146680,-0.184435,0.253668,-0.008810,0.125106,-0.092222,-0.078527,-0.150251,-0.051384
2,0.418252,-0.035081,-0.047205,-0.250219,0.083328,-0.018123,-0.144266,0.003115,0.110349,0.027673,...,-0.193988,0.114354,-0.423886,0.125146,0.272987,0.205443,0.052821,0.153954,0.095119,-0.438754
3,0.442539,0.218225,0.186937,-0.520042,0.150522,-0.214134,-0.091181,0.058205,0.075028,0.191456,...,-0.421022,-0.265187,-0.793915,0.220309,0.314230,0.159772,0.232910,0.334692,-0.097597,-0.356563
4,0.302827,0.053140,0.052276,-0.375106,0.071737,0.029059,-0.022829,-0.139187,0.074322,0.053337,...,-0.282669,0.147491,-0.326044,0.160345,0.144042,0.150934,-0.001184,0.076077,0.002166,-0.268126
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14375,0.170873,-0.002193,-0.162950,-0.251864,-0.072025,0.379080,-0.026370,0.242014,0.190190,0.201709,...,-0.267562,-0.160195,-0.259046,0.274229,0.183456,0.216945,-0.221591,-0.232861,-0.085996,-0.143322
14376,0.183422,0.270344,-0.066733,-0.287871,0.201977,0.122377,0.184230,-0.008778,0.187626,0.108932,...,-0.413719,0.271656,-0.129996,0.225421,-0.065121,0.092584,-0.079796,-0.062538,-0.159778,-0.017830
14377,0.200585,0.097627,-0.172556,-0.459374,-0.000506,0.196442,-0.052513,-0.022525,0.162465,0.157206,...,-0.186311,0.065225,-0.150048,0.212248,0.073744,0.159994,-0.160206,-0.168592,-0.345179,-0.085389
14378,0.191964,0.016449,-0.047402,-0.224906,0.026768,0.027088,-0.105560,0.171117,0.099281,0.160958,...,-0.255899,0.098224,-0.305825,0.066101,0.237134,0.231408,0.040428,-0.045997,0.067156,-0.344694


In [ ]:
# 계절 분리
df_train['sales_season'] = df_train['sales_month'].apply(lambda x : 'SPRING' if (x>=3) and (x<=5)
                                                  else 'SUMMER' if (x>=6) and (x<=8)
                                                  else 'FALL' if (x>=9) and (x<=11)
                                                  else 'WINTER')

df_test['sales_season'] = df_test['sales_month'].apply(lambda x : 'SPRING' if (x>=3) and (x<=5)
                                                  else 'SUMMER' if (x>=6) and (x<=8)
                                                  else 'FALL' if (x>=9) and (x<=11)
                                                  else 'WINTER')

In [ ]:
# 요일명을 영어로 변경 - lgbm에서 한글 피처 명이 있으면 안됨
df_train['sales_dayofweek'] = df_train['sales_dayofweek'].apply(lambda x : 'MONDAY' if x == '월'
                                                         else 'TUESDAY' if x == '화'
                                                         else 'WEDNESDAY' if x == '수'
                                                         else 'THURSDAY' if x == '목'
                                                         else 'FRIDAY' if x == '금'
                                                         else 'SATURDAY' if x == '토'
                                                         else 'SUNDAY')

df_test['sales_dayofweek'] = df_test['sales_dayofweek'].apply(lambda x : 'MONDAY' if x == '월'
                                                         else 'TUESDAY' if x == '화'
                                                         else 'WEDNESDAY' if x == '수'
                                                         else 'THURSDAY' if x == '목'
                                                         else 'FRIDAY' if x == '금'
                                                         else 'SATURDAY' if x == '토'
                                                         else 'SUNDAY')

In [ ]:
# 지점명을 영어로 변경
df_train['str_nm'] = df_train['str_nm'].apply(lambda x : 'MUYEOK_BRANCH' if x == '무역점'
                                                         else 'SHINCHON_BRANCH' if x == '신촌점'
                                                         else 'HEAD_OFFICE' if x == '본점'
                                                         else 'CHUNHO_BRANCH')

df_test['str_nm'] = df_test['str_nm'].apply(lambda x : 'MUYEOK_BRANCH' if x == '무역점'
                                                         else 'SHINCHON_BRANCH' if x == '신촌점'
                                                         else 'HEAD_OFFICE' if x == '본점'
                                                         else 'CHUNHO_BRANCH')

In [ ]:
# 시간대 구분 (회사원 기준)
df_train['time_split'] = df_train['sales_time'].apply(lambda x : 'MORNINGTIME' if (x>=700) and (x<=1130)
                                               else 'LUNCHTIME' if (x>1130) and (x<=1300)
                                               else 'AFTERNOON' if (x>1300) and (x<=1900)
                                               else 'RESTTIME')

df_test['time_split'] = df_test['sales_time'].apply(lambda x : 'MORNINGTIME' if (x>=700) and (x<=1130)
                                               else 'LUNCHTIME' if (x>1130) and (x<=1300)
                                               else 'AFTERNOON' if (x>1300) and (x<=1900)
                                               else 'RESTTIME')

In [ ]:
# 구매시간 정보
df_train['customer_info_time'] = df_train['str_nm'].astype(str) + '_' + df_train['sales_dayofweek'].astype(str) + '_' + df_train['sales_season'].astype(str) + '_' + df_train['time_split'].astype(str)

df_test['customer_info_time'] = df_test['str_nm'].astype(str) + '_' + df_test['sales_dayofweek'].astype(str) + '_' + df_test['sales_season'].astype(str) + '_' + df_test['time_split'].astype(str)

In [ ]:
import random

train_data = list(df_train.groupby('custid')['customer_info_time'].unique())
test_data = list(df_test.groupby('custid')['customer_info_time'].unique())

In [ ]:
w2v_input = oversample(train_data, 5)

In [ ]:
from gensim.models import word2vec

w2v = word2vec.Word2Vec(sentences = w2v_input, size = 100, window = 3, min_count = 1, sg = 1)

In [ ]:
train_mean_vector_4 = []
for words in tqdm(train_data):
    tmp = np.zeros(100)
    cnt = 0
    for word in words:
        try:
            tmp += w2v[word]
            cnt += 1
        except:
            pass
    tmp /= cnt
    train_mean_vector_4.append(tmp)
train_mean_vector_4 = np.array(train_mean_vector_4)

100%|██████████| 21587/21587 [00:02<00:00, 10633.89it/s]


In [ ]:
train_mean_vector_4 = pd.DataFrame(train_mean_vector_4)
train_mean_vector_4

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.032926,0.059582,-0.075069,0.229386,0.029044,0.146934,0.039504,-0.101144,-0.151681,0.024214,...,-0.051823,0.138253,-0.049987,-0.135735,0.074414,-0.066551,0.247540,0.287488,0.010650,0.036626
1,-0.025929,0.171175,-0.061845,0.087322,0.020285,0.161797,-0.067193,0.133736,-0.259073,0.148589,...,-0.071481,0.155945,0.165846,-0.204421,0.037730,0.115504,0.157027,0.232618,0.213876,0.171209
2,-0.082754,0.404789,-0.139013,-0.004859,-0.067835,0.286717,0.089990,0.071282,-0.135646,0.173876,...,-0.226109,0.111351,0.308292,-0.339873,0.042525,-0.035868,0.240484,0.054878,0.182061,0.221521
3,-0.164202,0.103500,0.020428,-0.049878,0.110556,0.249815,-0.038509,0.066457,-0.149728,0.118265,...,-0.025107,0.272885,0.204640,-0.207656,0.111977,0.109116,0.157788,0.207550,0.330932,0.210380
4,-0.158340,0.094741,-0.094345,0.171292,0.175992,0.320752,-0.185648,-0.148907,-0.092438,0.071396,...,-0.140166,0.232020,-0.069991,-0.114556,0.002995,-0.038132,0.268814,0.334329,-0.040407,0.172465
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21582,-0.149481,0.102297,-0.066180,0.030157,0.130133,0.246278,-0.117326,-0.046122,-0.129299,0.132000,...,-0.117995,0.194456,0.006229,-0.117202,0.072116,0.019823,0.228353,0.242027,0.186638,0.154973
21583,-0.047728,0.151030,-0.057562,0.047329,-0.066263,0.104183,-0.139472,0.128409,-0.175295,0.198893,...,-0.007022,0.217504,0.053358,-0.057045,0.043864,0.185687,0.186957,0.192919,0.228049,0.199307
21584,-0.080913,0.184035,-0.147436,0.197419,0.080955,0.170377,-0.196690,-0.086305,-0.034899,0.233154,...,-0.046428,0.137749,-0.104134,-0.105650,0.000077,-0.132684,0.206630,0.296944,0.063314,0.145858
21585,-0.060826,0.171792,0.032928,-0.060563,0.090983,0.148088,-0.053408,-0.002328,-0.168337,0.160171,...,-0.035857,0.240925,0.012232,-0.207275,0.063898,0.127905,0.154533,0.142881,0.247653,0.223105


In [ ]:
test_mean_vector_4 = []
for words in tqdm(test_data):
    tmp = np.zeros(100)
    cnt = 0
    for word in words:
        try:
            tmp += w2v[word]
            cnt += 1
        except:
            pass
    tmp /= cnt
    test_mean_vector_4.append(tmp)
test_mean_vector_4 = np.array(test_mean_vector_4)

100%|██████████| 14380/14380 [00:01<00:00, 10349.53it/s]


In [ ]:
test_mean_vector_4 = pd.DataFrame(test_mean_vector_4)
test_mean_vector_4

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.182461,0.092488,-0.129550,0.199635,0.294721,0.253667,-0.080115,-0.204279,-0.102860,0.119655,...,-0.056978,0.225241,-0.094711,-0.106935,0.056859,-0.100794,0.289048,0.338681,0.095316,0.118464
1,-0.080632,0.130160,-0.065420,0.053462,0.043414,0.178950,-0.117088,-0.034814,-0.131145,0.144174,...,-0.075228,0.218246,0.010022,-0.129622,0.045628,0.018943,0.239034,0.250504,0.130438,0.146311
2,-0.022235,0.166533,-0.099430,0.230510,0.396874,0.116745,-0.408290,-0.036446,0.206113,-0.104372,...,-0.004576,0.340747,0.102942,-0.175593,0.015396,0.082850,0.165382,0.264058,-0.484540,0.243069
3,-0.024169,0.023726,0.104056,0.303315,0.445945,0.216507,-0.422299,-0.107553,-0.093641,-0.385313,...,-0.000784,0.395174,-0.317195,-0.314298,0.223043,0.349206,0.125641,0.177511,-0.457658,0.110880
4,-0.062652,0.233393,-0.087092,0.177149,0.133451,-0.009900,-0.246665,0.001596,0.039956,-0.074558,...,-0.048729,0.201721,0.081405,-0.134189,-0.033070,0.150674,0.177427,0.176416,0.038701,0.196920
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14375,-0.223795,0.158545,-0.073159,0.297575,0.262752,0.209382,-0.187119,-0.182898,-0.118007,0.186092,...,0.026078,0.075702,-0.108263,-0.130195,-0.006575,-0.223479,0.166323,0.188265,0.015698,0.261456
14376,-0.187930,0.205949,0.070682,0.059726,-0.134679,0.129767,-0.313316,0.257381,-0.203148,0.212552,...,-0.113272,0.236953,0.002802,0.106227,0.109962,0.141610,0.108310,0.131585,0.379118,0.240654
14377,-0.156755,0.075451,-0.140859,0.158058,0.197632,0.241588,-0.063752,-0.209766,-0.210217,0.125433,...,-0.163650,0.137017,-0.085842,-0.137191,0.018620,0.007621,0.215714,0.363577,0.014270,0.079554
14378,0.007318,0.188806,-0.096672,0.254095,0.369109,0.054079,-0.442605,-0.037820,0.476857,-0.153508,...,0.058847,0.226791,-0.003735,-0.193242,0.008403,-0.033506,0.107038,0.319678,-0.398103,0.442289


In [ ]:
X_train = pd.concat([train_mean_vector, train_mean_vector_2], axis=1)
X_test = pd.concat([test_mean_vector, test_mean_vector_2], axis=1)

In [ ]:
X_train = pd.concat([X_train, train_mean_vector_3], axis=1)
X_test = pd.concat([X_test, test_mean_vector_3], axis=1)

In [ ]:
X_train = pd.concat([X_train, train_mean_vector_4], axis=1)
X_test = pd.concat([X_test, test_mean_vector_4], axis=1)

## feature selection

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/강민수+이효빈+한윤지/Input/snd_shap.0615.csv')
test = pd.read_csv('/content/drive/MyDrive/강민수+이효빈+한윤지/Input/snd_shap_te.0615.csv')

In [ ]:
train.shape

(21587, 1354)

In [ ]:
test.shape

(14380, 1354)

In [ ]:
train = train.iloc[:,2:]
test = test.iloc[:,2:]

In [ ]:
X_train.shape

(21587, 400)

In [ ]:
X_test.shape

(14380, 400)

In [ ]:
X_train = pd.concat([train, X_train], axis =1)
X_test = pd.concat([test, X_test], axis =1)

In [ ]:
X_train.shape

(21587, 1752)

In [ ]:
X_test.shape

(14380, 1752)

In [ ]:
X_all = pd.concat([X_train, X_test])
X_all.columns = list(pd.RangeIndex(len(X_all.columns)))

In [ ]:
X_all = X_all.loc[:,~X_all.T.duplicated()]

In [ ]:
X_train = X_all.iloc[:21587,:]
X_test = X_all.iloc[21587:,:]

In [ ]:
X_train.shape

(21587, 1752)

In [ ]:
X_test.shape

(14380, 1752)

In [ ]:
y_train = pd.read_csv('/content/drive/MyDrive/강민수+이효빈+한윤지/Input/y_train.csv').group

In [ ]:
import shap

In [ ]:
# DF, based on which importance is checked
X_importance = X_test

# Explain model predictions using shap library:
model = CatBoostClassifier(task_type = "GPU", random_state=42).fit(X_train, y_train)
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_importance)

shap_sum = np.abs(shap_values).mean(axis=1)[1,:]
importance_df = pd.DataFrame([X_importance.columns.tolist(), shap_sum.tolist()]).T
importance_df.columns = ['column_name', 'shap_importance']
importance_df = importance_df.sort_values('shap_importance', ascending=False)
importance_df

Learning rate set to 0.121467
0:	learn: 1.9505462	total: 51.4ms	remaining: 51.3s
1:	learn: 1.8663707	total: 94ms	remaining: 46.9s
2:	learn: 1.8040884	total: 133ms	remaining: 44.3s
3:	learn: 1.7577465	total: 172ms	remaining: 42.7s
4:	learn: 1.7207164	total: 211ms	remaining: 41.9s
5:	learn: 1.6912432	total: 248ms	remaining: 41.1s
6:	learn: 1.6658786	total: 286ms	remaining: 40.6s
7:	learn: 1.6432836	total: 326ms	remaining: 40.5s
8:	learn: 1.6242203	total: 365ms	remaining: 40.2s
9:	learn: 1.6083202	total: 404ms	remaining: 40s
10:	learn: 1.5943649	total: 442ms	remaining: 39.7s
11:	learn: 1.5814743	total: 481ms	remaining: 39.6s
12:	learn: 1.5697778	total: 520ms	remaining: 39.5s
13:	learn: 1.5607113	total: 557ms	remaining: 39.3s
14:	learn: 1.5517910	total: 596ms	remaining: 39.2s
15:	learn: 1.5437655	total: 636ms	remaining: 39.1s
16:	learn: 1.5354351	total: 675ms	remaining: 39s
17:	learn: 1.5290646	total: 714ms	remaining: 39s
18:	learn: 1.5231018	total: 752ms	remaining: 38.9s
19:	learn: 1.5169

,column_name,shap_importance
1,1.0,0.260533
0,0.0,0.208397
169,169.0,0.152720
231,231.0,0.143916
4,4.0,0.093751
...,...,...
1651,1651.0,0.000000
586,586.0,0.000000
1649,1649.0,0.000000
1460,1460.0,0.000000


In [ ]:
features_selected = importance_df.query('shap_importance > 0').column_name.tolist()
X_train = X_train[features_selected]
X_test = X_test[features_selected]
X_train.shape, X_test.shape

((21587, 1617), (14380, 1617))

In [ ]:
X_train.to_csv('/content/drive/MyDrive/강민수+이효빈+한윤지/Input/리얼리얼찐막_train.csv',index = False)
X_test.to_csv('/content/drive/MyDrive/강민수+이효빈+한윤지/Input/리얼리얼찐막_test.csv',index = False)

In [ ]:
#X_train = df.iloc[:21587]
#X_test = df.iloc[21587:]

# 사용할 모델 설정 (속도가 빠른 모델 사용 권장)
model = LGBMRegressor(random_state=0)

# 각 특성과 타깃(class) 사이에 유의한 통계적 관계가 있는지 계산하여 특성을 선택하는 방법 
cv_scores = []
for p in tqdm(range(5,100,3)):
    X_new = SelectPercentile(percentile=p).fit_transform(X_train, y_train)    
    cv_score = cross_val_score(model, X_new, y_train, scoring='neg_root_mean_squared_error', cv=5).mean()
    cv_scores.append((p,cv_score))

# Print the best percentile
best_score = cv_scores[np.argmax([score for _, score in cv_scores])]
print(best_score)

# Plot the performance change with p
plt.plot([k for k, _ in cv_scores], [score for _, score in cv_scores])
plt.xlabel('Percent of features')
plt.grid()

# 과적합을 피하기 위해 최적의 p값 주변의 값을 선택하는게 더 나은 결과를 얻을 수 있다. 
fs = SelectPercentile(percentile=best_score[0]).fit(X_train, y_train)
X_train = fs.transform(X_train)
X_test = fs.transform(X_test)

print(X_train.shape)

pd.DataFrame(X_train).to_csv('1,2등,우리_selection_3_W2V_2.csv')

pd.DataFrame(X_test).to_csv('1,2등,우리_selection_3_W2V_2_te.csv')